## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-21-06-58-41 +0000,bbc,Badenoch encourages Tory councils to challenge...,https://www.bbc.com/news/articles/cwy0j9n4qzwo...
1,2025-08-21-06-50-13 +0000,nypost,Fire on US warship off Japan injures 2 sailors...,https://nypost.com/2025/08/21/world-news/uss-n...
2,2025-08-21-06-38-53 +0000,nypost,Gymnast who sparked abuse inquiry into coach a...,https://nypost.com/2025/08/21/us-news/gymnast-...
3,2025-08-21-06-37-31 +0000,bbc,Government borrowing lower than expected in July,https://www.bbc.com/news/articles/cj6ydlk4e1no...
4,2025-08-21-06-20-54 +0000,nyt,Women in Indian Sports Sweat for Their Share o...,https://www.nytimes.com/2025/08/21/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2374/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
217,trump,42
99,new,21
7,fire,12
356,california,11
53,israel,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
138,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...,101
75,2025-08-21-00-19-00 +0000,wsj,President Trump has ordered the Pentagon to se...,https://www.wsj.com/politics/national-security...,88
99,2025-08-20-22-22-41 +0000,nyt,Trump Calls for Fed Governor Lisa Cook to Resi...,https://www.nytimes.com/2025/08/20/us/politics...,85
80,2025-08-21-00-03-43 +0000,bbc,Fed governor says she won't be 'bullied' as Tr...,https://www.bbc.com/news/articles/cx276gg2gwro...,83
113,2025-08-20-21-45-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...,82


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
138,101,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...
293,47,2025-08-20-10-00-00 +0000,latimes,"Amid Southern California heat wave, a fire bre...",https://www.latimes.com/california/story/2025-...
75,46,2025-08-21-00-19-00 +0000,wsj,President Trump has ordered the Pentagon to se...,https://www.wsj.com/politics/national-security...
128,45,2025-08-20-20-57-46 +0000,nypost,"JD Vance, Stephen Miller mock ‘old, white’ pro...",https://nypost.com/2025/08/20/us-news/jd-vance...
249,45,2025-08-20-14-01-00 +0000,wsj,Billionaire Bill Ackman has a new fascination:...,https://www.wsj.com/us-news/education/bill-ack...
185,44,2025-08-20-18-09-26 +0000,nypost,"Israel calls up 60,000 reservists for looming ...",https://nypost.com/2025/08/20/world-news/israe...
284,40,2025-08-20-10-00-00 +0000,latimes,Barabak: Power grab may energize Newsom and De...,https://www.latimes.com/politics/story/2025-08...
93,38,2025-08-20-22-40-00 +0000,wsj,Fed minutes from the July meeting reveal broad...,https://www.wsj.com/economy/central-banking/fe...
274,37,2025-08-20-11-59-04 +0000,nypost,Martha Nolan-O’Slatarra’s boyfriend gives hear...,https://nypost.com/2025/08/20/us-news/martha-n...
113,35,2025-08-20-21-45-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
